# Install / Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import edurel.utils.misc as mu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu


load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases/"

# Database

In [6]:
con = ddbu.mem_con(DB_DIR + "db-hvs")
schema = ddbu.schema(con)

In [4]:
# print(ddbu.schema(con))
# con.close()

# Queries

## q1

In [10]:
q1 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- pid, name aus tabelle person
- matrnr aus tabelle studentin
sortiert nach pid
"""

## q2

In [11]:
q2 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
- raumnr aus tabelle raum
für die studentin mit matrnr 1001
und die räume, in den sie kurse belegt hat
sortiert nach raumnr
"""

## q3

In [12]:
q3 = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
für alle studenten die im sommersemester 2023 kurse belegt haben,
die am montag stattfinden
"""

# ARCTICTEXT2SQL

In [19]:
arctic = llmu.ollama_c(llmu.ARCTICTEXT2SQL)

## q1

In [ ]:
mu.text2sql("q1", q1, arctic, schema, con)

In [ ]:
sql = """
"""

In [ ]:
ddbu.sql_print(con, sql)

# GLM46

In [8]:
glm46 = llmu.stats_c(llmu.GLM46)

In [16]:
mu.text2sql("q1", q1, glm46, schema, con)

q1:
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- pid, name aus tabelle person
- matrnr aus tabelle studentin
sortiert nach pid


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
ORDER BY p.pid;

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



In [17]:
mu.text2sql("q2", q2, glm46, schema, con)

q2:
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
- raumnr aus tabelle raum
für die studentin mit matrnr 1001
und die räume, in den sie kurse belegt hat
sortiert nach raumnr

SELECT p.name, s.matrnr, r.raumnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kb ON b.kursbt_kbtid = kb.kbtid
JOIN raum r ON kb.raum_rid = r.rid
WHERE s.matrnr = '1001'
ORDER BY r.raumnr;
┌────────────────┬─────────┬─────────┐
│      name      │ matrnr  │ raumnr  │
│    varchar     │ varchar │ varchar │
├────────────────┼─────────┼─────────┤
│ Max Mustermann │ 1001    │ A101    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ 

In [18]:
mu.text2sql("q3", q3, glm46, schema, con)

q3:
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
für alle studenten die im sommersemester 2023 kurse belegt haben,
die am montag stattfinden


SELECT p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kbt ON b.kursbt_kbtid = kbt.kbtid
JOIN kurs k ON kbt.kurs_kid = k.kid
JOIN semester sem ON k.semester_sid = sem.sid
JOIN termin t ON kbt.termin_tid = t.tid
JOIN wochentermin wt ON t.tid = wt.tid
JOIN tag tag ON wt.tag_tid = tag.tid
WHERE sem.bez = 'Sommersemester 2023'
AND tag.bez = 'Montag';

┌─────────┬─────────┐
│  name   │ matrnr  │
│ varchar │ varchar │
├─────────┴─────────┤
│      0 rows       │
└───────────────────┘

